# Environment
Load packages.

In [1]:
# Initialize workers
num_of_procs = 10 # You can also use addprocs() with no argument to create as many workers as your threads
using Distributed
addprocs(num_of_procs, exeflags="--project") # initial workers with the project env in current work directory

@everywhere push!(LOAD_PATH, "..")
using ParallelExp

# POMCPOW
@everywhere using POMCPOW
using BasicPOMCP

# LB-DESPOT
@everywhere push!(LOAD_PATH, "../../LB-DESPOT")
@everywhere using LBDESPOT # LBDESPOT pkg

# UCT-DESPOT
@everywhere push!(LOAD_PATH, "../../UCT-DESPOT")
@everywhere using UCTDESPOT # UCT-DESPOT pkg

# QMDP
using QMDP

# POMDP related pkgs
@everywhere using POMDPs # Basic POMDP framework
@everywhere using POMDPSimulators # For parallel simulator
using POMDPPolicies # For function policy and random policy
@everywhere using ParticleFilters # For simple particle filter
using BeliefUpdaters # For roomba and BabyPOMDP belief updater

# For visualization
using D3Trees
using POMDPModelTools
using POMDPGifs
import Cairo,Fontconfig

# For data processing and storing
using Statistics
using DataFrames
using CSV
using Random
using Printf

# RockSample Setting

In [ ]:
@everywhere using RockSample
pomdp = RockSamplePOMDP(map_size=(10,10),
                        rocks_positions=[(2,3), (1,8), (4,5), (5,2), (8,7)], 
                        sensor_efficiency=20.0,
                        discount_factor=0.95, 
                        good_rock_reward = 20.0)

# For LB-DESPOT
random_bounds = IndependentBounds(DefaultPolicyLB(RandomPolicy(pomdp)), 30.0, check_terminal=true)
lbdespot_dict = Dict(:default_action=>[RandomPolicy(pomdp),], 
                    :bounds=>[random_bounds],
                    :K=>[500, 300],
                    :beta=>[0.5, 0., 0.1, 1., 5.])

# For UCT-DESPOT
random_rollout_policy = RandomPolicy(pomdp)
uctdespot_dict = Dict(:default_action=>[RandomPolicy(pomdp),],
                    :rollout_policy=>[random_rollout_policy],
                    :max_trials=>[100000,],
                    :K=>[500, 1000],
                    :m=>[10, 30],
                    :c=>[10.,])

# For POMCPOW
random_value_estimator = FORollout(RandomPolicy(pomdp))
pomcpow_dict = Dict(:default_action=>[RandomPolicy(pomdp),],
                    :estimate_value=>[random_value_estimator],
                    :tree_queries=>[200000,], 
                    :max_time=>[1.0,], 
                    :criterion=>[MaxUCB(30.),])

# Solver list
solver_list = [LB_DESPOTSolver=>lbdespot_dict, 
                UCT_DESPOTSolver=>uctdespot_dict, 
                POMCPOWSolver=>pomcpow_dict,
                QMDPSolver=>Dict(:max_iterations=>[200,])]

# RockSample Visualization

In [ ]:
# solver = LB_DESPOTSolver(bounds=random_bounds, beta=0.5, K=100, default_action=RandomPolicy(pomdp))
# solver = UCT_DESPOTSolver(rollout_policy=random_rollout_policy, max_trials=100000, m=10, K=500, c=10)
# solver = POMCPOWSolver(tree_queries=200000, max_time=1.0, criterion=MaxUCB(30), estimate_value=random_value_estimator)

# planner = solve(solver, pomdp)
# makegif(pomdp, planner, filename="rock_sample.gif", max_steps=100, show_progress=true)

In [ ]:
# solver = LB_DESPOTSolver(bounds=random_bounds, beta=0.0, K=100, default_action=RandomPolicy(pomdp), tree_in_info=true)
# solver = UCT_DESPOTSolver(rollout_policy=random_rollout_policy, max_trials=100000, m=10, K=500, c=10, tree_in_info=true)
# solver = POMCPOWSolver(tree_queries=200000, max_time=1.0, criterion=MaxUCB(30), estimate_value=random_value_estimator, tree_in_info=true)

# planner = solve(solver, pomdp)
# b0 = initialstate(pomdp)
# a, info = action_info(planner, b0)
# println("number of trials: $(info[:record][1])")
# println("time for building DESPOT: $(info[:record][2])")
# D3Tree(info[:tree], init_expand=5)

# Tests based on RockSample

In [ ]:
number_of_episodes = 100
max_steps = 100

dfs = parallel_experiment(pomdp, number_of_episodes, max_steps, solver_list, full_factorial_design=false)

CSV.write("RockSample_LB-DESPOT.csv", dfs[1])
CSV.write("RockSample_UCT-DESPOT.csv", dfs[2])
CSV.write("RockSample_POMCPOW.csv", dfs[3])
CSV.write("RockSample_QMDP.csv", dfs[4])

# BabyPOMDP Setting
Setting up a BabyPOMDP problem for further using.

In [ ]:
@everywhere using POMDPModels # For BabyPOMDP
pomdp = BabyPOMDP(-5, -10, 0.1, 0.8, 0.1, 0.95) # defualt setting except that the discount is 0.95

# Feed When Crying Policy
@everywhere function feed_when_crying(b)
    if typeof(b) == Bool
        b 
    elseif (typeof(b) <: UCTDESPOT.ScenarioBelief || typeof(b) <: LBDESPOT.ScenarioBelief)
        if typeof(currentobs(b)) <: Bool
            currentobs(b)
        else
            pdf(currentobs(b), true) > 0.5
        end
    elseif typeof(b) <: POMDPModels.BoolDistribution
        rand(b)
    else
        pdf(b, true) > 0.5
    end
end
feed_when_crying_policy = solve(FunctionSolver(feed_when_crying), pomdp)

# For LB-DESPOT
# Assume all following rewards are coming from the worst case, "hungry but don't feed"
@everywhere fval(m::BabyPOMDP, x) = reward(m, true, false)/(1-discount(m))
bounds = IndependentBounds(DefaultPolicyLB(feed_when_crying_policy, final_value=fval), 0.0)
random_bounds = IndependentBounds(DefaultPolicyLB(RandomPolicy(pomdp), final_value=fval), 0.0)
lbdespot_dict = Dict(:default_action=>[feed_when_crying_policy,], 
                    :bounds=>[bounds, random_bounds],
                    :K=>[1000, 2000, 3600, 5000],
                    :beta=>[0., 0.1, 0.5, 1., 5.])

# For UCT-DESPOT
rollout_policy = feed_when_crying_policy
random_rollout_policy = RandomPolicy(pomdp)
uctdespot_dict = Dict(:rollout_policy=>[rollout_policy, random_rollout_policy],
                        :K=>[1000, 2000, 3600, 5000],
                        :m=>[30, 60, 100],
                        :c=>[500., 1000., 2000.])

# For POMCPOW
value_estimator = PORollout(feed_when_crying_policy, PreviousObservationUpdater())
random_value_estimator = FORollout(RandomPolicy(pomdp))
pomcpow_dict = Dict(:estimate_value=>[value_estimator, random_value_estimator],
                    :tree_queries=>[200000,], 
                    :max_time=>[1.0,], 
                    :criterion=>[MaxUCB(0.1), MaxUCB(1.0), MaxUCB(10.), MaxUCB(100.), MaxUCB(1000.)])

# Solver list
solver_list = [LB_DESPOTSolver=>lbdespot_dict, 
                UCT_DESPOTSolver=>uctdespot_dict, 
                POMCPOWSolver=>pomcpow_dict,
                QMDPSolver=>Dict(:max_iterations=>[200,]),
                FuncSolver=>Dict(:func=>[feed_when_crying,])]

# BabyPOMDP Visualization
Visualize BabyPOMDP in form of a tree.

In [ ]:
solver = LB_DESPOTSolver(bounds=bounds, beta=0.0, K=3000, default_action=feed_when_crying_policy, tree_in_info=true)
# solver = UCT_DESPOTSolver(rollout_policy=rollout_policy, m=1, K=1000, c=1000, tree_in_info=true)
# solver = POMCPOWSolver(tree_queries=200000, max_time=1.0, criterion=MaxUCB(1), estimate_value=value_estimator, tree_in_info=true)

planner = solve(solver, pomdp)
b0 = initialstate_distribution(pomdp)
a, info = action_info(planner, b0)
# println("number of trials: $(info[:record][1])")
# println("time for building DESPOT: $(info[:record][2])")
# println("tree depth: $(info[:record][3])")
D3Tree(info[:tree], init_expand=5)

# Tests based on BabyPOMDP
First, run a simulation of one episode and one max step in case there's some latent bugs.

In [ ]:
number_of_episodes = 1
max_steps = 1

dfs = parallel_experiment(pomdp, number_of_episodes, max_steps, solver_list, full_factorial_design=false)

# CSV.write("LaserTag_LB-DESPOT.csv", dfs[1])
CSV.write("LaserTag_UCT-DESPOT.csv", dfs[2])
# CSV.write("LaserTag_POMCPOW.csv", dfs[3])
# CSV.write("LaserTag_QMDP.csv", dfs[4])
# CSV.write("LaserTag_Move_Towards.csv", dfs[5])

# LaserTag Setting

In [ ]:
@everywhere using LaserTag
pomdp = gen_lasertag()
belief_updater = SIRParticleFilter(pomdp, 10000)

# Policies
@everywhere function move_towards(b)
    if typeof(b) <: LaserTag.LTInitialBelief
        return rand(1:9)
    elseif typeof(b) <: LBDESPOT.ScenarioBelief || 
        typeof(b) <: UCTDESPOT.ScenarioBelief || 
        typeof(b) <: ParticleFilters.ParticleCollection
        
        s = rand(b)
    else
        s = b
    end
    # try to sneak up diagonally
    diff = s.opponent-s.robot
    dx = diff[1]
    dy = diff[2]
    if abs(dx) == 1 && abs(dy) == 1
        LaserTag.DIR_TO_ACTION[[dx, dy]]
    elseif abs(dx) == 1
        LaserTag.DIR_TO_ACTION[[0, sign(dy)]]
    elseif abs(dy) == 1
        LaserTag.DIR_TO_ACTION[[sign(dx), 0]]
    else
        LaserTag.DIR_TO_ACTION[[sign(dx), sign(dy)]]
    end
end
move_towards_policy = solve(FunctionSolver(move_towards), pomdp)

# For LB-DESPOT
bounds = IndependentBounds(DefaultPolicyLB(move_towards_policy), 10.0, check_terminal=true)
random_bounds = IndependentBounds(DefaultPolicyLB(RandomPolicy(pomdp)), 10.0, check_terminal=true)
lbdespot_dict = Dict(:default_action=>[move_towards_policy,],
                    :min_reserve=>[5, 10],
                    :bounds=>[bounds, random_bounds],
                    :K=>[500, 1000, 3000, 5000],
                    :beta=>[0., 0.1, 0.5, 1., 5.])

# For UCT-DESPOT
rollout_policy = move_towards_policy
random_rollout_policy = RandomPolicy(pomdp)
uctdespot_dict = Dict(:rollout_policy=>[rollout_policy, random_rollout_policy],
                        :min_reserve=>[5, 10],
                        :K=>[500, 1000, 3000, 5000],
                        :m=>[1, 100],
                        :c=>[10., 20.])

# For POMCPOW
value_estimator = FORollout(move_towards_policy)
random_value_estimator = FORollout(RandomPolicy(pomdp))
pomcpow_dict = Dict(:estimate_value=>[value_estimator, random_value_estimator],
                    :tree_queries=>[150000,], 
                    :max_time=>[1.0,], 
                    :criterion=>[MaxUCB(500),MaxUCB(1000),MaxUCB(2000)])

# Solver list
solver_list = [LB_DESPOTSolver=>lbdespot_dict, 
                UCT_DESPOTSolver=>uctdespot_dict, 
                POMCPOWSolver=>pomcpow_dict,
                QMDPSolver=>Dict(:max_iterations=>[200,]),
                FuncSolver=>Dict(:func=>[move_towards,])]

# LaserTag Visualization

In [ ]:
# solver = LB_DESPOTSolver(bounds=bounds, beta=0, K=5000, tree_in_info=true)
# solver = UCT_DESPOTSolver(rollout_policy=rollout_policy, m=100, K=5000, c=10, tree_in_info=true)
solver = POMCPOWSolver(tree_queries=1000000, max_time=1.0, criterion=MaxUCB(1000), estimate_value=random_value_estimator, tree_in_info=true)

planner = solve(solver, pomdp)
b0 = initialstate_distribution(pomdp)
@time a, info = action_info(planner, b0)
# println("number of trials: $(first(info[:record]))")
# println("time for building DESPOT: $(last(info[:record]))")
D3Tree(info[:tree], init_expand=1)

# Tests based on LaserTag

In [ ]:
number_of_episodes = 1
max_steps = 1

dfs = parallel_experiment(pomdp, number_of_episodes, max_steps, solver_list, belief_updater=belief_updater, full_factorial_design=false)

CSV.write("LaserTag_LB-DESPOT.csv", dfs[1])
CSV.write("LaserTag_UCT-DESPOT.csv", dfs[2])
CSV.write("LaserTag_POMCPOW.csv", dfs[3])
CSV.write("LaserTag_QMDP.csv", dfs[4])
CSV.write("LaserTag_Move_Towards.csv", dfs[5])

# Roomba Env

In [2]:
# Roomba related pkgs
# Roomba need ParticleFilters = "0.2" for compatibility
@everywhere push!(LOAD_PATH, "../../Roomba")
@everywhere using Roomba # For Roomba Env

┌ Info: Precompiling Roomba [27451d18-47cd-4e6c-a8d6-4c3b695fab9f]
└ @ Base loading.jl:1260


# Bumper Roomba Setting
Setting up a Roomba problem with bumper sensor for further using.\
The parameters of Roomba are listed as follows.
```
maximum velocity of Roomba [m/s]
v_max::Float64  = 10.0  # m/s

maximum turn-rate of Roombda [rad/s]
om_max::Float64 = 1.0   # rad/s

simulation time-step [s]
dt::Float64     = 0.5   # s

penalty for wall-contact
contact_pen::Float64 = -1.0 

penalty per time-step
time_pen::Float64 = -0.1

reward for reaching goal
goal_reward::Float64 = 10

penalty for reaching stairs
stairs_penalty::Float64 = -10

specifies room configuration (location of stairs/goal) {1,2,3}
config::Int = 1

environment room struct
room::Room  = Room(sspace,configuration=config)

environment state-space (ContinuousRoombaStateSpace or DiscreteRoombaStateSpace)
sspace::SS = ContinuousRoombaStateSpace()

environment action-space struct
aspace::AS = RoombaActions()
```

In [ ]:
max_speed = 2.0
speed_interval = 2.0
max_turn_rate = 1.0
turn_rate_interval = 1.0

sensor = Bumper()
num_particles = 5000 # number of particles in belief

pos_noise_coeff = 0.3
ori_noise_coeff = 0.1

# POMDP problem
action_space = vec([RoombaAct(v, om) for v in 0:speed_interval:max_speed, om in -max_turn_rate:turn_rate_interval:max_turn_rate])
pomdp = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(aspace=action_space));

# Belief updater
resampler = BumperResampler(num_particles, pomdp, pos_noise_coeff, ori_noise_coeff)
belief_updater = BasicParticleFilter(pomdp, resampler, num_particles)

# Rush Policy
rush_policy = FunctionPolicy() do b
    if !(typeof(b) <: ParticleFilters.ParticleCollection) &&
        !(typeof(b) <: Roomba.RoombaInitialDistribution) &&
        b !== nothing &&
        typeof(b) == Bool ? b : (typeof(currentobs(b)) == Bool ? currentobs(b) : false)

        [max_speed, max_turn_rate]
    else
        [max_speed, 0.0]
    end
end

# For LB-DESPOT
bounds = IndependentBounds(DefaultPolicyLB(rush_policy), 10.0, check_terminal=true)
random_bounds = IndependentBounds(DefaultPolicyLB(RandomPolicy(pomdp)), 10.0, check_terminal=true)
lbdespot_dict = Dict(:default_action=>[rush_policy,], 
                    :bounds=>[bounds, random_bounds],
                    :K=>[100, 300, 500],
                    :beta=>[0., 0.1, 1., 10., 100.])

# For UCT-DESPOT
rollout_policy = rush_policy
random_rollout_policy = RandomPolicy(pomdp)
uctdespot_dict = Dict(:rollout_policy=>[rollout_policy, random_rollout_policy],
                        :K=>[100, 300, 500],
                        :m=>[5, 10, 20, 30],
                        :c=>[0.1, 1., 10., 100., 1000., 10000.])

# For POMCPOW
value_estimator = PORollout(rush_policy, PreviousObservationUpdater())
random_value_estimator = FORollout(RandomPolicy(pomdp))
pomcpow_dict = Dict(:estimate_value=>[value_estimator, random_value_estimator],
                    :tree_queries=>[100000,], 
                    :max_time=>[1.0,], 
                    :criterion=>[MaxUCB(0.1), MaxUCB(1.0), MaxUCB(10.), MaxUCB(100.), MaxUCB(1000.)])

# Solver list
solver_list = [LB_DESPOTSolver=>lbdespot_dict, 
                UCT_DESPOTSolver=>uctdespot_dict, 
                POMCPOWSolver=>pomcpow_dict]

# Bumper Roomba Visualization

In [ ]:
# solver = LB_DESPOTSolver(bounds=bounds, default_action=rush_policy, tree_in_info=true)
solver = UCT_DESPOTSolver(rollout_policy=rollout_policy, tree_in_info=true)
# solver = POMCPOWSolver(tree_queries=100000, max_time=1.0, estimate_value=value_estimator, tree_in_info=true)

planner = solve(solver, pomdp)
makegif(pomdp, planner, belief_updater, filename="bumper.gif", max_steps=100, show_progress=true)

In [ ]:
# solver = LB_DESPOTSolver(bounds=bounds, beta=0.2, T_max=60.0, K=100, default_action=rush_policy, tree_in_info=true)
# solver = UCT_DESPOTSolver(rollout_policy=rollout_policy, T_max=60.0, m=10, K=100, c=100, tree_in_info=true)
# # solver = POMCPOWSolver(tree_queries=1000000, max_time=20.0, criterion=MaxUCB(1000), estimate_value=value_estimator, tree_in_info=true)

# planner = solve(solver, pomdp)
# b0 = initialstate_distribution(pomdp)
# @time a, info = action_info(planner, b0)
# @show info[:record]
# D3Tree(info[:tree], init_expand=5)

# Tests based on Bumper Roomba

In [ ]:
number_of_episodes = 1
max_steps = 1

dfs = parallel_experiment(pomdp, number_of_episodes, max_steps, solver_list, belief_updater=belief_updater)
for i in 1:length(dfs)
    CSV.write("BumperRoomba$(i).csv", dfs[i])
end

# Lidar Roomba Setting

In [ ]:
max_speed = 2.0
speed_interval = 2.0
max_turn_rate = 1.0
turn_rate_interval = 1.0

sensor = Lidar()
num_particles = 5000 # number of particles in belief

pos_noise_coeff = 0.3
ori_noise_coeff = 0.1

# POMDP problem
action_space = vec([RoombaAct(v, om) for v in 0:speed_interval:max_speed, om in -max_turn_rate:turn_rate_interval:max_turn_rate])
pomdp = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=2, aspace=action_space))

# Belief updater
resampler = LidarResampler(num_particles, pomdp, pos_noise_coeff, ori_noise_coeff)
belief_updater = BasicParticleFilter(pomdp, resampler, num_particles)

# Running policy
running_policy = FunctionPolicy() do b
    # s = typeof(b) == RoombaState ? b : typeof(b) <: AA228FinalProject.RoombaInitialDistribution ? rand(b) : mean(b)
    # The statement is computational inefficient.
    s = typeof(b) == RoombaState ? b : rand(b)
    # compute the difference between our current heading and one that would
    # point to the goal
    goal_x, goal_y = get_goal_xy(pomdp)
    x,y,th = s[1:3]
    ang_to_goal = atan(goal_y - y, goal_x - x)
    del_angle = wrap_to_pi(ang_to_goal - th)
    
    # apply proportional control to compute the turn-rate
    Kprop = 1.0
    om = Kprop * del_angle
    # find the closest option in action space
    _,ind = findmin(abs.(om .- (-max_turn_rate:turn_rate_interval:max_turn_rate)))
    om = (-max_turn_rate:turn_rate_interval:max_turn_rate)[ind]
    # always travel at some fixed velocity
    v = max_speed
    
    return RoombaAct(v, om)
end

# For LB-DESPOT
bounds = IndependentBounds(DefaultPolicyLB(running_policy), 10.0, check_terminal=true)
random_bounds = IndependentBounds(DefaultPolicyLB(RandomPolicy(pomdp)), 10.0, check_terminal=true)
lbdespot_dict = Dict(:default_action=>[running_policy,], 
                    :bounds=>[bounds, random_bounds],
                    :lambda=>[0.0, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0],
                    :K=>[100, 300, 500],
                    :beta=>[0., 0.1, 1., 10., 100.])

# For UCT-DESPOT
rollout_policy = running_policy
random_rollout_policy = RandomPolicy(pomdp)
uctdespot_dict = Dict(:rollout_policy=>[rollout_policy, random_rollout_policy],
                        :K=>[100, 300, 500],
                        :m=>[5, 10, 20, 30],
                        :criterion=>[MaxUCB(0.1), MaxUCB(1.0), MaxUCB(10.), MaxUCB(100.), MaxUCB(1000.)])

# For POMCPOW
value_estimator = FORollout(running_policy)
random_value_estimator = FORollout(RandomPolicy(pomdp))
pomcpow_dict = Dict(:estimate_value=>[value_estimator, random_value_estimator],
                    :tree_queries=>[100000,], 
                    :max_time=>[1.0,], 
                    :c=>[0.1, 1., 10., 100., 1000., 10000.])

# Solver list
solver_list = [LB_DESPOTSolver=>lbdespot_dict, 
                UCT_DESPOTSolver=>uctdespot_dict, 
                POMCPOWSolver=>pomcpow_dict]

# Lidar Roomba Visualization

In [ ]:
# solver = LB_DESPOTSolver(bounds=bounds, default_action=running_policy)
# solver = UCT_DESPOTSolver(rollout_policy=rollout_policy)
# solver = POMCPOWSolver(tree_queries=100000, max_time=1.0, estimate_value=value_estimator)

# planner = solve(solver, pomdp)
# makegif(pomdp, planner, belief_updater, filename="lidar.gif", max_steps=100, show_progress=true)

In [ ]:
# solver = LB_DESPOTSolver(bounds=bounds, default_action=running_policy, tree_in_info=true)
# solver = UCT_DESPOTSolver(rollout_policy=rollout_policy, tree_in_info=true)
# solver = POMCPOWSolver(tree_queries=100000, max_time=1.0, estimate_value=value_estimator, tree_in_info=true)

# planner = solve(solver, pomdp)
# b0 = initialstate_distribution(pomdp)
# a, info = action_info(planner, b0)
# D3Tree(info[:tree], init_expand=5)

# Tests based on Lidar Roomba

In [ ]:
number_of_episodes = 1
max_steps = 1

dfs = parallel_experiment(pomdp, number_of_episodes, max_steps, solver_list, belief_updater=belief_updater)
for i in 1:length(dfs)
    CSV.write("LidarRoomba$(i).csv", dfs[i])
end

# Discrete Lidar Roomba

In [3]:
max_speed = 2.0
speed_interval = 2.0
max_turn_rate = 1.0
turn_rate_interval = 1.0

cut_points =  exp10.(range(-.5, stop=1.3, length=10))
sensor = DiscreteLidar(cut_points)

num_particles = 5000 # number of particles in belief

pos_noise_coeff = 0.3
ori_noise_coeff = 0.1

# POMDP problem
action_space = vec([RoombaAct(v, om) for v in 0:speed_interval:max_speed, om in -max_turn_rate:turn_rate_interval:max_turn_rate])
pomdp = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=3, aspace=action_space));

# Belief updater
resampler = LidarResampler(num_particles, pomdp, pos_noise_coeff, ori_noise_coeff)
belief_updater = BasicParticleFilter(pomdp, resampler, num_particles)

# Running policy
running_policy = FunctionPolicy() do b
    # s = typeof(b) == RoombaState ? b : typeof(b) <: AA228FinalProject.RoombaInitialDistribution ? rand(b) : mean(b)
    # The statement is computational inefficient.
    s = typeof(b) == RoombaState ? b : rand(b)
    # compute the difference between our current heading and one that would
    # point to the goal
    goal_x, goal_y = get_goal_xy(pomdp)
    x,y,th = s[1:3]
    ang_to_goal = atan(goal_y - y, goal_x - x)
    del_angle = wrap_to_pi(ang_to_goal - th)
    
    # apply proportional control to compute the turn-rate
    Kprop = 1.0
    om = Kprop * del_angle
    # find the closest option in action space
    _,ind = findmin(abs.(om .- (-max_turn_rate:turn_rate_interval:max_turn_rate)))
    om = (-max_turn_rate:turn_rate_interval:max_turn_rate)[ind]
    # always travel at some fixed velocity
    v = max_speed
    
    return RoombaAct(v, om)
end

# For LB-DESPOT
bounds = IndependentBounds(DefaultPolicyLB(running_policy), 10.0, check_terminal=true)
random_bounds = IndependentBounds(DefaultPolicyLB(RandomPolicy(pomdp)), 10.0, check_terminal=true)
lbdespot_dict = Dict(:default_action=>[running_policy,], 
                    :bounds=>[bounds, random_bounds],
                    :lambda=>[0.0, 0.01, 0.1, 1.0],
                    :T_max=>[30.0],
                    :K=>[100, 500],
                    :beta=>[0., 0.5, 1., 5.])

# For UCT-DESPOT
rollout_policy = running_policy
random_rollout_policy = RandomPolicy(pomdp)
uctdespot_dict = Dict(:rollout_policy=>[rollout_policy, random_rollout_policy],
                        :K=>[100, 500],
                        :T_max=>[30.0],
                        :m=>[10, 50],
                        :c=>[1., 10., 100.])

# For POMCPOW
value_estimator = FORollout(running_policy)
random_value_estimator = FORollout(RandomPolicy(pomdp))
pomcpow_dict = Dict(:estimate_value=>[value_estimator, random_value_estimator],
                    :tree_queries=>[100000,], 
                    :max_time=>[30.0,], 
                    :criterion=>[MaxUCB(0.1), MaxUCB(1.0), MaxUCB(10.), MaxUCB(100.), MaxUCB(1000.)])

# Solver list
solver_list = [LB_DESPOTSolver=>lbdespot_dict, 
                UCT_DESPOTSolver=>uctdespot_dict, 
                POMCPOWSolver=>pomcpow_dict]

3-element Array{Pair{Type,Dict{Symbol,Array{T,1} where T}},1}:
  LB_DESPOTSolver => Dict(:lambda => [0.0, 0.01, 0.1, 1.0],:default_action => [FunctionPolicy{var"#5#6"}(var"#5#6"())],:beta => [0.0, 0.5, 1.0, 5.0],:K => [100, 500],:T_max => [30.0],:bounds => IndependentBounds{L,Float64} where L[IndependentBounds{DefaultPolicyLB{FunctionPolicy{var"#5#6"},Nothing,LBDESPOT.var"#23#25"},Float64}(DefaultPolicyLB{FunctionPolicy{var"#5#6"},Nothing,LBDESPOT.var"#23#25"}(FunctionPolicy{var"#5#6"}(var"#5#6"()), nothing, LBDESPOT.var"#23#25"()), 10.0, true, 0.0), IndependentBounds{DefaultPolicyLB{RandomPolicy{Random._GLOBAL_RNG,RoombaPOMDP{DiscreteLidar,Int64},NothingUpdater},Nothing,LBDESPOT.var"#23#25"},Float64}(DefaultPolicyLB{RandomPolicy{Random._GLOBAL_RNG,RoombaPOMDP{DiscreteLidar,Int64},NothingUpdater},Nothing,LBDESPOT.var"#23#25"}(RandomPolicy{Random._GLOBAL_RNG,RoombaPOMDP{DiscreteLidar,Int64},NothingUpdater}(Random._GLOBAL_RNG(), RoombaPOMDP{DiscreteLidar,Int64}(DiscreteLidar(0.1, [0.3162

# Discrete Lidar Visualization

In [ ]:
solver = LB_DESPOTSolver(bounds=bounds, beta=0.2, K=500, default_action=running_policy)
# solver = UCT_DESPOTSolver(m=10, K=100, c=100, T_max=1, rollout_policy=rollout_policy)
# solver = POMCPOWSolver(tree_queries=100000, max_time=1.0, estimate_value=value_estimator)

planner = solve(solver, pomdp)
makegif(pomdp, planner, belief_updater, filename="discrete_lidar.gif", max_steps=100, show_progress=true)

In [8]:
# solver = LB_DESPOTSolver(bounds=bounds, T_max=30.0,lambda=1, beta=0.2, K=300, default_action=running_policy, tree_in_info=true)
solver = UCT_DESPOTSolver(rollout_policy=rollout_policy, T_max=10.0, m=10, K=500, c=1, tree_in_info=true)
# solver = POMCPOWSolver(tree_queries=100000, max_time=30.0, criterion=MaxUCB(10), estimate_value=value_estimator, tree_in_info=true)

planner = solve(solver, pomdp)
b0 = initialstate_distribution(pomdp)
a, info = action_info(planner, b0)
println("number of trials: $(first(info[:record]))")
# @show info[:record]
println("time for building DESPOT: $(last(info[:record]))")
D3Tree(info[:tree], init_expand=2)

number of trials: 209.0
time for building DESPOT: 10.109


o:<root> (|Φ|:500)
N:214
├──a:[0.0, -1.0]
│  V: -0.49, N:120
│  ├──o:7 (|Φ|: 73)
│  │  N:21
│  │  ├──a:[0.0, -1.0]
│  │  │  V:  0.07, N:12 (8 children)
│  │  ├──a:[2.0, -1.0]
│  │  │  V: -1.44, N:2 (6 children)
│  │  ├──a:[0.0, 0.0]
│  │  │  V: -0.75, N:2 (2 children)
│  │  ├──a:[2.0, 0.0]
│  │  │  V: -2.17, N:1 (3 children)
│  │  ├──a:[0.0, 1.0]
│  │  │  V: -1.28, N:2 (6 children)
│  │  └──a:[2.0, 1.0]
│  │     V: -2.13, N:2 (7 children)
│  ├──o:10 (|Φ|: 71)
│  │  N:19
│  │  ├──a:[0.0, -1.0]
│  │  │  V: -1.35, N:2 (5 children)
│  │  ├──a:[2.0, -1.0]
│  │  │  V: -0.38, N:4 (6 children)
│  │  ├──a:[0.0, 0.0]
│  │  │  V: -0.63, N:2 (3 children)
│  │  ├──a:[2.0, 0.0]
│  │  │  V: -0.06, N:5 (3 children)
│  │  ├──a:[0.0, 1.0]
│  │  │  V: -0.83, N:2 (5 children)
│  │  └──a:[2.0, 1.0]
│  │     V: -0.10, N:4 (7 children)
│  ├──o:11 (|Φ|: 45)
│  │  N:17
│  │  ├──a:[0.0, -1.0]
│  │  │  V: -1.46, N:2 (5 children)
│  │  ├──a:[2.0, -1.0]
│  │  │  V: -1.09, N:2 (5 children)
│  │  ├──a:[0.0, 0.0]
│  │  │  V: -0.06, N:5 (2 children)
│  │  ├──a:[2.0, 0.0]
│  │  │  V: -0.75, N:2 (2 children)
│  │  ├──a:[0.0, 1.0]
│  │  │  V: -0.35, N:3 (6 children)
│  │  └──a:[2.0, 1.0]
│  │     V: -0.62, N:3 (6 children)
│  ├──o:8 (|Φ|: 92)
│  │  N:25
│  │  ├──a:[0.0, -1.0]
│  │  │  V: -1.54, N:2 (5 children)
│  │  ├──a:[2.0, -1.0]
│  │  │  V: -0.34, N:9 (8 children)
│  │  ├──a:[0.0, 0.0]
│  │  │  V: -0.90, N:3 (3 children)
│  │  ├──a:[2.0, 0.0]
│  │  │  V: -0.61, N:5 (4 children)
│  │  ├──a:[0.0, 1.0]
│  │  │  V: -1.62, N:2 (4 children)
│  │  └──a:[2.0, 1.0]
│  │     V: -0.55, N:4 (7 children)
│  ├──o:9 (|Φ|:117)
│  │  N:37
│  │  ├──a:[0.0, -1.0]
│  │  │  V: -1.59, N:2 (5 children)
│  │  ├──a:[2.0, -1.0]
│  │  │  V:  0.01, N:9 (6 children)
│  │  ├──a:[0.0, 0.0]
│  │  │  V:  0.02, N:6 (3 children)
│  │  ├──a:[2.0, 0.0]
│  │  │  V:  0.06, N:14 (3 children)
│  │  ├──a:[0.0, 1.0]
│  │  │  V: -0.33, N:5 (5 children)
│  │  └──a:[2.0, 1.0]
│  │     V: -2.10, N:1 (4 children)
│  ├──o:6 (|Φ|: 58)
│  │  N:20
│  │  ├──a:[0.0, -1.0]
│  │  │  V:  0.56, N:2 (6 children)
│  │  ├──a:[2.0, -1.0]
│  │  │  V:  0.44, N:5 (7 children)
│  │  ├──a:[0.0, 0.0]
│  │  │  V: -0.39, N:2 (3 children)
│  │  ├──a:[2.0, 0.0]
│  │  │  V: -0.58, N:1 (3 children)
│  │  ├──a:[0.0, 1.0]
│  │  │  V:  0.66, N:9 (6 children)
│  │  └──a:[2.0, 1.0]
│  │     V: -0.40, N:1 (6 children)
│  ├──o:5 (|Φ|: 26)
│  │  N:17
│  │  ├──a:[0.0, -1.0]
│  │  │  V: -1.36, N:2 (4 children)
│  │  ├──a:[2.0, -1.0]
│  │  │  V: -0.49, N:5 (7 children)
│  │  ├──a:[0.0, 0.0]
│  │  │  V: -1.22, N:3 (3 children)
│  │  ├──a:[2.0, 0.0]
│  │  │  V: -1.73, N:2 (4 children)
│  │  ├──a:[0.0, 1.0]
│  │  │  V: -0.56, N:3 (7 children)
│  │  └──a:[2.0, 1.0]
│  │     V: -0.69, N:2 (8 children)
│  ├──o:4 (|Φ|: 14)
│  │  N:8
│  │  ├──a:[0.0, -1.0]
│  │  │  V: -1.55, N:3 (4 children)
│  │  ├──a:[2.0, -1.0]
│  │  │  V: -2.75, N:1 (4 children)
│  │  ├──a:[0.0, 0.0]
│  │  │  V: -2.72, N:1 (2 children)
│  │  ├──a:[2.0, 0.0]
│  │  │  V: -2.81, N:1 (3 children)
│  │  ├──a:[0.0, 1.0]
│  │  │  V: -2.98, N:1 (5 children)
│  │  └──a:[2.0, 1.0]
│  │     V: -2.91, N:1 (5 children)
│  └──o:3 (|Φ|:  4)
│     N:0
├──a:[2.0, -1.0]
│  V: -0.71, N:40
│  ├──o:7 (|Φ|: 57)
│  │  N:9
│  │  ├──a:[0.0, -1.0]
│  │  │  V: -0.05, N:2 (6 children)
│  │  ├──a:[2.0, -1.0]
│  │  │  V:  0.36, N:3 (6 children)
│  │  ├──a:[0.0, 0.0]
│  │  │  V: -0.65, N:1 (3 children)
│  │  ├──a:[2.0, 0.0]
│  │  │  V: -1.17, N:1 (2 children)
│  │  ├──a:[0.0, 1.0]
│  │  │  V: -0.54, N:1 (5 children)
│  │  └──a:[2.0, 1.0]
│  │     V: -1.39, N:1 (4 children)
│  ├──o:10 (|Φ|: 46)
│  │  N:13
│  │  ├──a:[0.0, -1.0]
│  │  │  V: -1.00, N:2 (5 children)
│  │  ├──a:[2.0, -1.0]
│  │  │  V: -0.38, N:4 (5 children)
│  │  ├──a:[0.0, 0.0]
│  │  │  V: -1.53, N:2 (3 children)
│  │  ├──a:[2.0, 0.0]
│  │  │  V: -1.24, N:1 (2 children)
│  │  ├──a:[0.0, 1.0]
│  │  │  V: -1.06, N:1 (5 children)
│  │  └──a:[2.0, 1.0]
│  │     V: -0.61, N:3 (7 children)
│  ├──o:11 (|Φ|: 35)
│  │  N:9
│  │  ├──a:[0.0, -1.0]
│  │  │

# Tests based on Discrete Lidar Roomba

In [ ]:
number_of_episodes = 1
max_steps = 1

dfs = parallel_experiment(pomdp, number_of_episodes, max_steps, solver_list, belief_updater=belief_updater, full_factorial_design=false)

CSV.write("DiscreteLidarRoomba_LB_DESPOT.csv", dfs[1])
CSV.write("DiscreteLidarRoomba_UCT_DESPOT.csv", dfs[2])
CSV.write("DiscreteLidarRoomba_POMCPOW.csv", dfs[3])